In [1]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import cPickle as pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
  model = pickle.load(open('save.p', 'rb'))
else:
  model = {}
  model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
  model['W2'] = np.random.randn(H) / np.sqrt(H)
  

In [2]:
print model['W1'].shape
print model['W2'].shape

(200, 6400)
(200,)


In [3]:
grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems() } # rmsprop memory

In [4]:
print grad_buffer['W1'].shape, grad_buffer['W2'].shape
print rmsprop_cache['W1'].shape, rmsprop_cache['W2'].shape

(200, 6400) (200,)
(200, 6400) (200,)


### 기타 호출되는 함수들

In [5]:
def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()

### feed forward

In [6]:
def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state


### feed backward

In [7]:
def policy_backward(eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}


### 여기서부터 메인 함수

In [8]:
env = gym.make("Pong-v0")
observation = env.reset()

[2016-07-15 19:12:53,266] Making new env: Pong-v0


In [9]:
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

### 한번에 테스트

In [10]:
def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(xrange(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r


In [ ]:
import pdb

while True:
  #if render: env.render()

  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x

  # forward the policy network and sample an action from the returned probability
  aprob, h = policy_forward(x)
  action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state
  y = 1 if action == 2 else 0 # a "fake label"
  dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

  # step the environment and get new measurements
  observation, reward, done, info = env.step(action)
  reward_sum += reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    episode_number += 1

        
    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    eph = np.vstack(hs)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    
    print len(xs), len(hs), len(dlogps), len(drs)
    print epx.shape, eph.shape, epdlogp.shape, epr.shape
    
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr -= np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)

    #pdb.set_trace()

    epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
    print 'discounted_epr:',discounted_epr.shape
    
    # 여기서부터는 policy_backward를 테스트
    print '======================='
    print eph.shape, epdlogp.shape
    dW2 = np.dot(eph.T, epdlogp)
    print 'dW2:',dW2.shape
    dW2 = dW2.ravel()
    print dW2.shape
    
    dh = np.outer(epdlogp, model['W2'])
    print dh.shape, model['W2'].shape
    
    dh[eph <= 0] = 0 # backpro prelu
    print 'dh:',dh.shape, 'epx:', epx.shape
    dW1 = np.dot(dh.T, epx)
    print dW1.shape, dW2.shape
    
    grad = {'W1':dW1, 'W2':dW2} #policy_backward(eph, epdlogp)
    
    for k in model: 
        print k
        grad_buffer[k] += grad[k] 
    
    
        
    pdb.set_trace()
    
    

1492 1492 1492 1492
(1492, 6400) (1492, 200) (1492, 1) (1492, 1)
discounted_epr: (1492, 1)
(1492, 200) (1492, 1)
dW2: (200, 1)
(200,)
(1492, 200) (200,)
dh: (1492, 200) epx: (1492, 6400)
(200, 6400) (200,)
W2
W1
> <ipython-input-11-1a984c2e5b37>(3)<module>()
-> while True:


### 여기는 메인 루프를 하나하나 실행해 봄

In [11]:
print observation.shape
cur_x = prepro(observation)
print cur_x.shape

(210, 160, 3)
(6400,)


In [12]:
x = cur_x - prev_x if prev_x is not None else np.zeros(D)
prev_x = cur_x


In [13]:
aprob, h = policy_forward(x)
print aprob, h.shape

0.5 (200,)


In [14]:
action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

In [15]:
xs.append(x) # observation
hs.append(h) # hidden state
y = 1 if action == 2 else 0 # a "fake label"
dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)


In [16]:
print y, dlogps
print xs, len(xs), len(hs)

0 [-0.5]
[array([ 0.,  0.,  0., ...,  0.,  0.,  0.])] 1 1


In [17]:
observation, reward, done, info = env.step(action)
reward_sum += reward


In [18]:
print observation.shape, reward, done, info

(210, 160, 3) 0.0 False {}
